In [ ]:
# need to get the word embeddings for the data used, and save it in embeddings.txt

# use first 1.0-rw-preProcessText_makeBOW or1.0-rw-preProcessText_makeBOW_forTopicAssignment to pre-process, 
# and now something similar to skipgram.py 
# (but without creating word embeddings, instead use pre-trained ones, eg these from FLAiR)
# to create the embeddings.txt file

In [1]:
import gensim
import pickle
import os
import numpy as np
import argparse

from flair.embeddings import WordEmbeddings
from flair.data import Sentence

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
parser = argparse.ArgumentParser(description='The Embedded Topic Model')

### data and file related arguments
parser.add_argument('--data_file', type=str, default='data/corpus.txt', help='a .txt file containing the corpus')
parser.add_argument('--emb_file', type=str, default='data/embeddings.txt', help='file to save the word embeddings')
parser.add_argument('--dim_rho', type=int, default=300, help='dimensionality of the word embeddings')
parser.add_argument('--min_count', type=int, default=2, help='minimum term frequency (to define the vocabulary)')
parser.add_argument('--sg', type=int, default=1, help='whether to use skip-gram')
parser.add_argument('--workers', type=int, default=25, help='number of CPU cores')
parser.add_argument('--negative_samples', type=int, default=10, help='number of negative samples')
parser.add_argument('--window_size', type=int, default=4, help='window size to determine context')
parser.add_argument('--iters', type=int, default=50, help='number of iterationst')


_StoreAction(option_strings=['--iters'], dest='iters', nargs=None, const=None, default=50, type=<class 'int'>, choices=None, help='number of iterationst', metavar=None)

In [3]:
args, unknown = parser.parse_known_args()

# args = parser.parse_args()
# based on solution from:
# https://stackoverflow.com/questions/48796169/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter

In [4]:
# init embedding
word_embedding = WordEmbeddings('crawl')

In [5]:
######

# Class for a memory-friendly iterator over the dataset
class MyEmbeddings(object):
    def __init__(self, filename, w_e):
        self.filename = filename
        self.word_embedding = w_e
 
    # Write the embeddings to a file
    def print_2_file(self):
        with open(args.emb_file, 'w') as f:
            for line in open(self.filename):
                newLine = Sentence(line.replace("\n", " "))
                
                # embed sentence using crawl.
                self.word_embedding.embed(newLine)
                
                l = len(newLine)
                for i in range(0, l):
                    token = newLine[i] 
                    repStr= "Token: " + str(i+1)
                    v = str(token).replace(repStr, "").replace(" ", "")
                    vec = token.embedding
                    f.write(v + ' ')
                    vec_str = ['%.9f' % val for val in vec]
                    vec_str = " ".join(vec_str)
                    f.write(vec_str + '\n')
        return None


In [6]:
sentences = MyEmbeddings(filename = args.data_file, w_e = word_embedding) # a memory-friendly iterator
sentences.print_2_file()